In [1]:
import json
import networkx as nx 
from networkx.readwrite import json_graph
import pandas as pd
from IPython.display import display, Math, Latex
import re
import sys
sys.path.append("D:\\Users\\figohjs\\Documents\\Network")
from Script.helper_v2 import *
from Script.generateHtml_v4 import generateHtml

### UDF

### read data

In [4]:
#json file - subnetwork
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2021-02-19_ScamNetwork52Names.json"
with open(jsonFile, 'r') as f:
    tempGraphDict = json.load(f)

csvFile = "D:/Users/figohjs/Documents/Network/data/raw/subject.csv"
df = pd.read_csv(csvFile)
offenseList = df['Offence'].unique()

### Overview

In [5]:
#How many person in scam in repo
scamOffense = [i for i in offenseList if re.search("Scam", str(i), re.I)]
dfScam = df[df['Offence'].isin(scamOffense)].copy()
numScamOffense = dfScam.shape[0]
print("Number of scam cases: %s"%numScamOffense)
print("Missing ID: %s"%dfScam[pd.isnull(dfScam['ID'])].shape[0])
print("Number of unique names: %s"%len(dfScam['Name'].unique()))

Number of scam cases: 73
Missing ID: 73
Number of unique names: 52


In [6]:
#clean name
dfScam['CleanName'] = dfScam['Name'].replace(to_replace = ['\(.*\)', '\.'],
                                             value = '',  regex = True).str.strip()
scamNameList  = dfScam['CleanName'].unique()

#build neoId to name dict
neoIdtoNeoNameDict = {node['NeoId']:node['pName']
                     for node in tempGraphDict['nodes'] if node['label'] in ['Entity']}
neoIdtoNameDictTargetOnly = {i:j for i,j in neoIdtoNeoNameDict.items() if j in scamNameList}

#one name may have more than 1 id
print("Number of unique names found in network: %s"%len(set(neoIdtoNameDictTargetOnly.values())))

Number of unique names found in network: 16


In [6]:
print("Number of unique nodes: %s"%len(tempGraphDict['nodes']))
print("Number of links: %s"%len(tempGraphDict['links']))

Number of unique nodes: 48481
Number of links: 96387


In [7]:
#build network graph from dict of dict
networkGraph = json_graph.node_link_graph(tempGraphDict)
#find number of subgraphs
subGraphs = nx.connected_components(networkGraph)
#filter out subgraphs with only one node
subGraphsList = [i for i in list(subGraphs) if len(i)!=1]
print("Number of subgraphs: %s"%len(list(subGraphsList)))
assert len(list(subGraphsList)) == 43, "number of subgraphs should be 43"

Number of subgraphs: 43


In [37]:
[i for i in tempGraphDict['nodes'] if i['NeoId'] == 8209078]

[{'uid': 'e-25e67746-6fcd-421e-974d-e5402e733792',
  'pOccupation': ['Lawyer / Barrister / Solicitor'],
  'pName': 'MOHD FAIZAL BIN AHMAD',
  'pNameList': ['MOHD FAIZAL BIN AHMAD'],
  'pCountry': ['MALAYSIA'],
  'pIdList': ['700520016143', 'A1576955'],
  'contact_list': ['072217828', '0127777828', '073337929', '072217929'],
  'pOccupationDesc': [],
  'label': 'Entity',
  'NeoId': 8209078}]

In [38]:
[i for i in tempGraphDict['nodes'] if i['NeoId'] == 8288836]

[{'uid': 'e-b37e6a4d-8fa7-4508-b06e-b8815b838d21',
  'pOccupation': [],
  'pName': 'MOHD FAIZAL BIN AHMAD',
  'pNameList': ['MOHD FAIZAL BIN AHMAD'],
  'pCountry': ['MALAYSIA'],
  'pIdList': ['840727085669'],
  'contact_list': ['0122553104', '058072323', '0166613702', '016613702'],
  'pOccupationDesc': [],
  'label': 'Entity',
  'NeoId': 8288836}]

In [39]:
[i for i in tempGraphDict['nodes'] if i['NeoId'] == 8412685]

[{'uid': 'e-f7923333-87b3-465c-b0c1-fd1805969e1d',
  'pOccupation': ['Lawyer / Barrister / Solicitor'],
  'pName': 'MOHD FAIZAL BIN AHMAD',
  'pNameList': ['MOHD FAIZAL BIN AHMAD', 'MOHD FAIZAL'],
  'pCountry': ['MALAYSIA'],
  'pIdList': ['730803145049', 'A2445711'],
  'contact_list': ['00387360515',
   '0122902314',
   '27308541',
   '0387360515',
   '0192902314',
   'MFAIZAL@UDANET.CONM'],
  'pOccupationDesc': [],
  'label': 'Entity',
  'NeoId': 8412685}]

In [9]:
#build neoId to es uid dict
neoIdtoUidDict = {node['NeoId']:node['uid']
                 for node in tempGraphDict['nodes'] if node['label'] in ['Entity']}

#neoId to Str Amount Dict
neoIdtoStrAmountDict = dict([(i['NeoId'], i['pTotalAmount']) for i in tempGraphDict['nodes'] if i['label'] == 'Account'])

#neoId to Ctr Amount Dict
accountInfo = [i for i in tempGraphDict['nodes'] if i['label'] == 'Account']
yearCol = ['p' + str(i) for i in range(2006, 2021)]
neoIdtoCtrAmountDict = dict([(i['NeoId'], sum([float(i[col]) for col in i.keys() if col in yearCol])) for i in accountInfo])

#neoIds of str node
strNodeNeoIds = [i['NeoId'] for i in tempGraphDict['nodes'] if i['label'] == 'STR']

#details of str node
strNodeDetails = [i for i in tempGraphDict['nodes'] if i['label'] == 'STR']

In [45]:
strNodeDetails[0]

{'uid': 'r-37f1d0c9-a54d-49ab-8b72-60a050f17c23',
 'pOffence': 'Money Laundering',
 'pStateId': 'JOHOR',
 'pTrxAmount': '763798.81',
 'pReportDate': '20/02/2014',
 'pAttempted': 'N',
 'pRecordTypeID': 'STR financial',
 'pToTrxDate': '28/01/2014',
 'pBranchId': 'JOHOR BAHRU (CITY SQUARE)',
 'pRecordID': 'AA/021/S/2014/000199',
 'pFromTrxDate': '23/02/2012',
 'label': 'STR',
 'NeoId': 14548206}

### Build table - validation purpose

In [16]:
dfNetworkScam = pd.DataFrame()

#list of neoIds for all nodes for each network
dfNetworkScam['NeoIds'] = [i for i in subGraphsList]

#get all transaction id for each network
dfNetworkScam['TransIds'] = dfNetworkScam['NeoIds'].map(lambda x:[i['transId'] for i in tempGraphDict['links'] 
                                            if i['source'] in x and i['target'] in x])

In [17]:
#check if str connected to each network contains scam/fraud
def checkIfScam(neoIdArray):
    offenceList = []
    for array in neoIdArray:
        tempList = []
        strList = set(array) & set(strNodeNeoIds)
        for strId in strList:
            tempList.append([i['pOffence'] for i in strNodeDetails if i['NeoId'] == strId][0])
        offenceList.append(tempList)
    #check if scam/fraud in related str
#     print(offenceList)
    scamList = []
    for offence in offenceList:
        temp = [True for i in offence if re.search("scam|fraud", str(i), re.I)]
        if len(temp)>0:
            scamList.append(True)
        else:
            scamList.append(False)
    return scamList

dfNetworkScam['IsScamFraud'] = checkIfScam(dfNetworkScam['NeoIds'].values)

#target neoIds
dfNetworkScam['TargetNeoIds'] = dfNetworkScam['NeoIds'].map(lambda x:set([i for i in x if i in neoIdtoNameDictTargetOnly.keys()]))

#target names
dfNetworkScam['TargetNames'] = dfNetworkScam['TargetNeoIds'].map(lambda x:[neoIdtoNameDictTargetOnly[i] for i in x])

In [22]:
nameList = dfNetworkScam[dfNetworkScam['IsScamFraud']]['TargetNames']
uniqueNameList = set([j for i in nameList for j in i])
print("Number of names found in network related to scam/fraud: %s"%len(uniqueNameList))
print("Number of network related to scam/fraud: %s"%sum(dfNetworkScam['IsScamFraud']))

Number of names found in network related to scam/fraud: 15
Number of network related to scam/fraud: 10


In [66]:
#the only name left out
set(neoIdtoNameDictTargetOnly.values()).difference(uniqueNameList)

{'NORZERIN BINTI MISKAM'}

In [68]:
tempDf = dfNetworkScam[dfNetworkScam['TargetNames'].map(lambda x: True if 'NORZERIN BINTI MISKAM' in x else False)].copy()
tempDf

,NeoIds,TransIds,IsScamFraud,TargetNeoIds,TargetNames
14,"{16950017, 11893795, 16137924, 16929763, 41719...","[11566431, 11566430, 19286470, 19286469, 14859...",False,{11109587},[NORZERIN BINTI MISKAM]


In [69]:
#check if str connected to each network contains scam/fraud
def checkIfScam(neoIdArray):
    offenceList = []
    for array in neoIdArray:
        tempList = []
        strList = set(array) & set(strNodeNeoIds)
        for strId in strList:
            tempList.append([i['pOffence'] for i in strNodeDetails if i['NeoId'] == strId][0])
        offenceList.append(tempList)
    #check if scam/fraud in related str
    print(offenceList)
checkIfScam(tempDf['NeoIds'].values)

[[]]


In [77]:
for nodeId in tempDf['NeoIds'].values[0]:
    print([i for i in tempGraphDict['nodes'] if i['NeoId'] == nodeId][0])
    print('\n')

{'uid': 'c-1726b7ef-e393-4b11-8874-df6848bf0da4', 'pContactInfo': '0033723333', 'label': 'Contact', 'NeoId': 16950017}


{'uid': 'e-fa38a8c4-1114-4bc6-9625-377f12064102', 'pOccupation': ['Police Officer'], 'pName': 'MONA ARI', 'pNameList': ['MONA ARI'], 'pCountry': ['MALAYSIA'], 'pIdList': ['RF130701', '770104135358', 'K0387972'], 'contact_list': ['00355145280', '0193749693', '0197539912', '0355145231', '0176236761'], 'pOccupationDesc': [], 'label': 'Entity', 'NeoId': 11893795}


{'uid': 'c-3a32a00a-9203-43ba-8ed5-87190931d320', 'pContactInfo': '0140091546770000', 'label': 'Contact', 'NeoId': 16137924}


{'uid': 'c-c3079f9e-6043-4b9b-9666-075c86fe9b40', 'pContactInfo': '0176236761', 'label': 'Contact', 'NeoId': 16929763}


{'uid': 'a-f5ae56df-cb2a-4622-a96b-d6f769ddda66', 'pTrxTypeId': 'Aggregate', 'pTotalAmount': 89937.2, 'pAccountNo': '0740020009396', 'pStateId': 'SELANGOR', 'pAccountType': 'Savings Account', 'pReportingIns': 'AmIslamic Bank Berhad.', 'p2008': 89937.2, 'pBranchId': '

In [89]:
#extract ids
targetNeoIdList = [j for i in dfNetworkScam[dfNetworkScam['IsScamFraud']]['TargetNeoIds'] for j in i]
tempIdList = [i['pIdList'] for i in tempGraphDict['nodes'] if i['NeoId'] in targetNeoIdList]
targetIdList = [j for i in tempIdList for j in i]
#export to csv
import csv
filename = "D:/Users/figohjs/Documents/Network/data/interim/2021-03-03_ScamId.csv"
# with open(filename, "w") as f:
#     writer = csv.writer(f)
#     writer.writerows(targetNeoIdList)
pd.Series(targetIdList).to_csv(filename, index = False)

### General Characteristics

In [18]:
# dfNetworkScam = pd.DataFrame()

# #list of neoIds for all nodes for each network
# dfNetworkScam['NeoIds'] = [i for i in subGraphsList]

# #get all transaction id for each network
# dfNetworkScam['TransIds'] = dfNetworkScam['NeoIds'].map(lambda x:[i['transId'] for i in tempGraphDict['links'] 
#                                             if i['source'] in x and i['target'] in x])

#number of nodes for each network
dfNetworkScam['NumNodes'] = [len(i) for i in subGraphsList]

#number of connection for each network
dfNetworkScam['NumConnection'] = [len(i) - 1 for i in subGraphsList]

#calculate number of Str
dfNetworkScam['NumStr'] = dfNetworkScam['NeoIds'].map(lambda x:len(set(x) & set(strNodeNeoIds)))

#get strAmount for each neoIds and sum it for each network
dfNetworkScam['StrAmount'] = dfNetworkScam['NeoIds'].map(lambda x:sum([neoIdtoStrAmountDict.get(i, 0) for i in x]))

#get ctrAmount for each neoIds and sum it for each network
dfNetworkScam['CtrAmount'] = dfNetworkScam['NeoIds'].map(lambda x:sum([neoIdtoCtrAmountDict.get(i, 0) for i in x]))

#target neoIds
dfNetworkScam['TargetNeoIds'] = dfNetworkScam['NeoIds'].map(lambda x:set([i for i in x if i in neoIdtoNameDictTargetOnly.keys()]))

#target names
dfNetworkScam['TargetNames'] = dfNetworkScam['TargetNeoIds'].map(lambda x:[neoIdtoNameDictTargetOnly[i] for i in x])

#number of targets for each network based on unique id
dfNetworkScam['NumberTargetNeoId'] = dfNetworkScam['TargetNeoIds'].map(lambda x:len(set(x)))

#number of targets for each network based on unique names
dfNetworkScam['NumberTargetNames'] = dfNetworkScam['TargetNames'].map(lambda x:len(set(x)))

#check if target shares contact for each network
contactTransInfo = [i for i in tempGraphDict['links'] if i['type'] == 'HAS_CONTACT'] 
def checkTargetSharingContact(transIdArray, targetNeoIdArray):
    resultBool = []
    for i in zip(transIdArray, targetNeoIdArray):
        #loop through contact transID if it is found in transID for each row
        targetArray = [trans['target'] for trans in contactTransInfo 
                       if trans['transId'] in i[0] and trans['source'] in i[1]]
        
        #1 if has shared contact else 0
        if len(set(targetArray)) != len(targetArray):
            resultBool.append(1)
        else:
            resultBool.append(0)          
    return resultBool

dfNetworkScam['TargetSharingContact'] = checkTargetSharingContact(dfNetworkScam['TransIds'].values,
                                                                  dfNetworkScam['TargetNeoIds'].values)

### Network perspective

In [13]:
Latex(r"""
\begin{equation}
Density = \frac{C}{n(n-1)}
\end{equation}
\begin{align}
\text{where } &\text{C - actual connection,}
\newline
&\text{n - number of nodes}
\end{align}
""")

<IPython.core.display.Latex object>

In [19]:
def calculateDensity(x):
    numCon = x['NumConnection']
    numNode = x['NumNodes']
    return numCon/(numNode*(numNode -1))

#calculate density for each network
dfNetworkScam['Density'] = dfNetworkScam.apply(calculateDensity, axis = 1)

#degree centrality
#assumption: important node has many connection d(i) = number of edges/no.of nodes - 1
neoIdToDC = nx.degree_centrality(networkGraph)

#degree centrality for each node for each network
dfNetworkScam['DCList'] = dfNetworkScam['NeoIds'].map(lambda x:[neoIdToDC.get(i,0) for i in x])

#get max degree centrality for each network
dfNetworkScam['MaxDC'] = dfNetworkScam['DCList'].map(max)

In [20]:
dfNetworkScam.columns

Index(['NeoIds', 'TransIds', 'IsScamFraud', 'TargetNeoIds', 'TargetNames',
       'NumNodes', 'NumConnection', 'NumStr', 'StrAmount', 'CtrAmount',
       'NumberTargetNeoId', 'NumberTargetNames', 'TargetSharingContact',
       'Density', 'DCList', 'MaxDC'],
      dtype='object')

In [21]:
#save df
filename = "D:/Users/figohjs/Documents/Network/data/interim/2021-03-09_ScamSubgraphsDF.csv"
dfNetworkScam.to_csv(filename, index = False)